In [1]:
import os

In [2]:
%pwd

'd:\\Opencv_projects\\chest_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Opencv_projects\\chest_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.CNNChestClassifier.constants import *
from src.CNNChestClassifier.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [13]:
import urllib.request as request
import os
import gdown
import zipfile
from src.CNNChestClassifier import logger
from src.CNNChestClassifier.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self,config : DataIngestionConfig):
        self.config = config

    def download_data(self):
        '''
        Fetch data fro mthe url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artificats/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    
    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-15 02:20:38,225: INFO: common: yaml file: config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_mo

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW&confirm=t&uuid=a2ff1026-9276-4bb9-8668-2e33bf796b29
To: d:\Opencv_projects\chest_classification\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:03<00:00, 13.6MB/s]

[2024-07-15 02:20:47,163: INFO: 2479113722: Downloaded data from https://drive.google.com/file/d/1ooEK-e37Ig48zPBvi27d2AY9lhN34EPW/view?usp=sharing into file artifacts/data_ingestion/data.zip]
